# Sudan Geospatial Analysis with DuckDB

This notebook demonstrates geospatial analysis and map visualization using the **DuckDB Sudan Extension**.

We'll create interactive and static maps combining data from 5 international APIs with real polygon boundaries (GADM v4.1) for Sudan's 18 states.

**Maps included:**
- Base map of Sudan's 18 states (bilingual labels)
- Population time series & regional comparison
- Displacement & refugees (UNHCR)
- Health indicators (WHO)
- Agricultural production (FAO)
- Interactive Folium map with popups
- State area analysis from polygon geometries

## 1. Setup & Installation

In [ ]:
%pip install duckdb==1.4.4 geopandas folium matplotlib mapclassify branca shapely --force-reinstall --no-cache-dir

> **Important:** After running the cell above, go to **Runtime -> Restart runtime** before continuing.

In [ ]:
import duckdb
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import json
from shapely.geometry import shape
from branca.colormap import linear

print(f"DuckDB version: {duckdb.__version__}")

# Connect and load extension
conn = duckdb.connect(config={'allow_unsigned_extensions': 'true'})
conn.execute("INSTALL httpfs; LOAD httpfs;")
conn.execute("SET custom_extension_repository = 'https://osman-geomatics93.github.io/duckdb-sudan-';")
conn.execute("INSTALL sudan; LOAD sudan;")

# Verify
conn.sql("SELECT * FROM SUDAN_Providers()").show()
print("\nSudan extension loaded successfully!")

## 2. Load Boundaries into GeoDataFrame

The extension embeds real MultiPolygon boundaries (GADM v4.1) for all 18 states. No network needed for boundaries.

In [ ]:
# Fetch state boundaries with bilingual names
states_df = conn.sql("""
    SELECT state_name, state_name_ar, iso_code, centroid_lon, centroid_lat, geojson
    FROM SUDAN_States()
""").df()

# Parse GeoJSON strings into Shapely geometries
states_df['geometry'] = states_df['geojson'].apply(lambda g: shape(json.loads(g)))
gdf = gpd.GeoDataFrame(states_df, geometry='geometry', crs='EPSG:4326')
gdf = gdf.drop(columns=['geojson'])

print(f"Loaded {len(gdf)} states")
gdf[['state_name', 'state_name_ar', 'iso_code']].head()

In [ ]:
# Also load country outline
country_df = conn.sql("""
    SELECT country_name, geojson FROM SUDAN_Boundaries('country')
""").df()
country_df['geometry'] = country_df['geojson'].apply(lambda g: shape(json.loads(g)))
country_gdf = gpd.GeoDataFrame(country_df, geometry='geometry', crs='EPSG:4326')
country_gdf = country_gdf.drop(columns=['geojson'])

## 3. Base Map — Sudan's 18 States

A static map showing all 18 states with English and Arabic labels.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 14))

# Plot states with distinct colors
gdf.plot(ax=ax, column='state_name', cmap='Set3', edgecolor='black', linewidth=0.5, legend=False)

# Add labels
for idx, row in gdf.iterrows():
    ax.annotate(
        f"{row['state_name']}\n{row['state_name_ar']}",
        xy=(row['centroid_lon'], row['centroid_lat']),
        ha='center', va='center', fontsize=7, fontweight='bold',
        bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7, edgecolor='none')
    )

ax.set_title("Sudan's 18 States\n\u0648\u0644\u0627\u064a\u0627\u062a \u0627\u0644\u0633\u0648\u062f\u0627\u0646 \u0627\u0644\u0640 18", fontsize=16, fontweight='bold')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

## 4. Population Analysis (World Bank)

Sudan's population over time and comparison with neighboring countries.

In [ ]:
# Fetch population data for Sudan and neighbors
pop_df = conn.sql("""
    SELECT country_name, year, value
    FROM SUDAN_WorldBank('SP.POP.TOTL', countries := ['SDN', 'EGY', 'ETH', 'TCD', 'SSD', 'ERI', 'LBY', 'CAF'])
    WHERE value IS NOT NULL AND year >= 1990
    ORDER BY country_name, year
""").df()

print(f"Fetched {len(pop_df)} rows")
pop_df.head()

In [ ]:
# Population time series for all countries
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Left: Line chart - population over time
for country in pop_df['country_name'].unique():
    data = pop_df[pop_df['country_name'] == country]
    lw = 3 if country == 'Sudan' else 1.2
    axes[0].plot(data['year'], data['value'] / 1e6, label=country, linewidth=lw)

axes[0].set_title('Population Over Time (1990-present)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Population (millions)')
axes[0].legend(loc='upper left', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Right: Bar chart - latest year comparison
latest_year = pop_df['year'].max()
latest = pop_df[pop_df['year'] == latest_year].sort_values('value', ascending=True)
colors = ['#e74c3c' if c == 'Sudan' else '#3498db' for c in latest['country_name']]
axes[1].barh(latest['country_name'], latest['value'] / 1e6, color=colors)
axes[1].set_title(f'Population Comparison ({latest_year})', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Population (millions)')
for i, (_, row) in enumerate(latest.iterrows()):
    axes[1].text(row['value'] / 1e6 + 0.5, i, f"{row['value']/1e6:.1f}M", va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 5. Displacement & Refugees (UNHCR)

Sudan is experiencing one of the world's largest displacement crises.

In [ ]:
# Fetch refugee and IDP data
refugees_df = conn.sql("""
    SELECT year, population_type, country_origin_name, country_asylum_name, value
    FROM SUDAN_UNHCR('refugees')
    WHERE value > 0
    ORDER BY year DESC, value DESC
""").df()

idps_df = conn.sql("""
    SELECT year, value
    FROM SUDAN_UNHCR('idps')
    WHERE value > 0
    ORDER BY year
""").df()

print(f"Refugees: {len(refugees_df)} rows, IDPs: {len(idps_df)} rows")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Left: IDPs over time
if len(idps_df) > 0:
    axes[0].bar(idps_df['year'], idps_df['value'] / 1e6, color='#e74c3c', alpha=0.8)
    axes[0].set_title('Internally Displaced Persons (IDPs)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Year')
    axes[0].set_ylabel('People (millions)')
    axes[0].grid(True, alpha=0.3, axis='y')

# Right: Top asylum countries (latest year)
if len(refugees_df) > 0:
    latest_ref_year = refugees_df['year'].max()
    top_asylum = refugees_df[refugees_df['year'] == latest_ref_year].nlargest(10, 'value')
    axes[1].barh(top_asylum['country_asylum_name'], top_asylum['value'] / 1e3, color='#2980b9', alpha=0.8)
    axes[1].set_title(f'Sudanese Refugees by Asylum Country ({latest_ref_year})', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('People (thousands)')
    for i, (_, row) in enumerate(top_asylum.iterrows()):
        axes[1].text(row['value'] / 1e3 + 1, i, f"{row['value']:,.0f}", va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 6. Health Indicators (WHO)

Maternal mortality ratio trend from the WHO Global Health Observatory.

In [ ]:
# Maternal mortality ratio
mmr_df = conn.sql("""
    SELECT year, value
    FROM SUDAN_WHO('MDG_0000000026')
    WHERE value IS NOT NULL
    ORDER BY year
""").df()

# Life expectancy at birth
le_df = conn.sql("""
    SELECT year, sex, value
    FROM SUDAN_WHO('WHOSIS_000001')
    WHERE value IS NOT NULL
    ORDER BY year, sex
""").df()

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Left: Maternal mortality
if len(mmr_df) > 0:
    axes[0].plot(mmr_df['year'], mmr_df['value'], 'o-', color='#e74c3c', linewidth=2, markersize=6)
    axes[0].fill_between(mmr_df['year'], mmr_df['value'], alpha=0.15, color='#e74c3c')
    axes[0].set_title('Maternal Mortality Ratio (per 100,000 live births)', fontsize=13, fontweight='bold')
    axes[0].set_xlabel('Year')
    axes[0].set_ylabel('Deaths per 100,000')
    axes[0].grid(True, alpha=0.3)
else:
    axes[0].text(0.5, 0.5, 'No maternal mortality data available', ha='center', va='center', transform=axes[0].transAxes)

# Right: Life expectancy by sex
if len(le_df) > 0:
    for sex in le_df['sex'].unique():
        data = le_df[le_df['sex'] == sex]
        axes[1].plot(data['year'], data['value'], 'o-', label=sex, linewidth=2, markersize=5)
    axes[1].set_title('Life Expectancy at Birth (years)', fontsize=13, fontweight='bold')
    axes[1].set_xlabel('Year')
    axes[1].set_ylabel('Years')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
else:
    axes[1].text(0.5, 0.5, 'No life expectancy data available', ha='center', va='center', transform=axes[1].transAxes)

plt.tight_layout()
plt.show()

## 7. Agriculture & Food Security (FAO)

Top crops produced in Sudan from FAOSTAT.

In [ ]:
# Crop production
crops_df = conn.sql("""
    SELECT item, year, value, unit
    FROM SUDAN_FAO('QCL', 'production')
    WHERE value IS NOT NULL AND value > 0
    ORDER BY year DESC, value DESC
""").df()

print(f"Fetched {len(crops_df)} rows")

if len(crops_df) > 0:
    # Latest year top crops
    latest_crop_year = crops_df['year'].max()
    top_crops = crops_df[crops_df['year'] == latest_crop_year].nlargest(10, 'value')

    fig, axes = plt.subplots(1, 2, figsize=(18, 7))

    # Left: Top crops bar chart
    axes[0].barh(top_crops['item'], top_crops['value'] / 1e3, color='#27ae60', alpha=0.8)
    axes[0].set_title(f'Top Crops by Production ({latest_crop_year})', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Production (thousand tonnes)')
    axes[0].invert_yaxis()

    # Right: Top 5 crops over time
    top5_items = top_crops.head(5)['item'].tolist()
    for item in top5_items:
        data = crops_df[crops_df['item'] == item].sort_values('year')
        axes[1].plot(data['year'], data['value'] / 1e3, 'o-', label=item, linewidth=2, markersize=4)

    axes[1].set_title('Top 5 Crops Production Over Time', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Year')
    axes[1].set_ylabel('Production (thousand tonnes)')
    axes[1].legend(fontsize=9)
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()
else:
    print('No crop data available')

## 8. State Regions Map

Classify Sudan's 18 states into geographic regions and visualize as a choropleth.

In [ ]:
# Define regions
region_map = {
    'North Darfur': 'Darfur', 'South Darfur': 'Darfur', 'West Darfur': 'Darfur',
    'East Darfur': 'Darfur', 'Central Darfur': 'Darfur',
    'North Kordofan': 'Kordofan', 'South Kordofan': 'Kordofan', 'West Kordofan': 'Kordofan',
    'Khartoum': 'Central', 'Al Jazirah': 'Central', 'White Nile': 'Central',
    'Blue Nile': 'Central', 'Sennar': 'Central',
    'Kassala': 'Eastern', 'Al Qadarif': 'Eastern', 'Red Sea': 'Eastern',
    'River Nile': 'Northern', 'Northern': 'Northern'
}

gdf['region'] = gdf['state_name'].map(region_map)

region_colors = {
    'Darfur': '#e74c3c', 'Kordofan': '#f39c12', 'Central': '#2ecc71',
    'Eastern': '#3498db', 'Northern': '#9b59b6'
}
gdf['color'] = gdf['region'].map(region_colors)

fig, ax = plt.subplots(1, 1, figsize=(12, 14))
for region, color in region_colors.items():
    subset = gdf[gdf['region'] == region]
    subset.plot(ax=ax, color=color, edgecolor='black', linewidth=0.5, label=region)

# Add state labels
for _, row in gdf.iterrows():
    ax.annotate(
        row['state_name'], xy=(row['centroid_lon'], row['centroid_lat']),
        ha='center', va='center', fontsize=7, fontweight='bold',
        bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7, edgecolor='none')
    )

ax.legend(title='Region', fontsize=11, title_fontsize=12, loc='lower left')
ax.set_title('Sudan - Geographic Regions', fontsize=16, fontweight='bold')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

## 9. State Area Analysis

Compute approximate area of each state from polygon geometries and create an area choropleth map.

In [ ]:
# Compute approximate area in km2 (using a projected CRS for accuracy)
gdf_proj = gdf.to_crs(epsg=32636)  # UTM Zone 36N (covers Sudan)
gdf['area_km2'] = gdf_proj.geometry.area / 1e6  # m2 to km2

fig, axes = plt.subplots(1, 2, figsize=(18, 14))

# Left: Choropleth map by area
gdf.plot(ax=axes[0], column='area_km2', cmap='YlOrRd', edgecolor='black', linewidth=0.5,
         legend=True, legend_kwds={'label': 'Area (km\u00b2)', 'shrink': 0.5})
for _, row in gdf.iterrows():
    axes[0].annotate(
        f"{row['state_name']}\n{row['area_km2']:,.0f} km\u00b2",
        xy=(row['centroid_lon'], row['centroid_lat']),
        ha='center', va='center', fontsize=6,
        bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7, edgecolor='none')
    )
axes[0].set_title('State Area (km\u00b2)', fontsize=14, fontweight='bold')
axes[0].set_aspect('equal')

# Right: Horizontal bar chart
sorted_gdf = gdf.sort_values('area_km2', ascending=True)
colors = plt.cm.YlOrRd(sorted_gdf['area_km2'] / sorted_gdf['area_km2'].max())
axes[1].barh(sorted_gdf['state_name'], sorted_gdf['area_km2'] / 1e3, color=colors)
axes[1].set_title('State Area Comparison', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Area (thousand km\u00b2)')
for i, (_, row) in enumerate(sorted_gdf.iterrows()):
    axes[1].text(row['area_km2'] / 1e3 + 0.5, i, f"{row['area_km2']:,.0f}", va='center', fontsize=8)

plt.tight_layout()
plt.show()

print(f"\nTotal area: {gdf['area_km2'].sum():,.0f} km\u00b2")

## 10. Interactive Folium Map

An interactive map with popups showing state details. Click on any state to see its name (English/Arabic), ISO code, region, and area.

In [ ]:
# Create base map centered on Sudan
m = folium.Map(location=[15.5, 30.0], zoom_start=6, tiles='cartodbpositron')

# Color by region
for _, row in gdf.iterrows():
    geojson = json.loads(row['geometry'].__geo_interface__.__str__().replace("'", '"'))
    
    popup_html = f"""
    <div style="font-family: Arial; width: 200px;">
        <h4 style="margin:0;">{row['state_name']}</h4>
        <h4 style="margin:0; direction:rtl;">{row['state_name_ar']}</h4>
        <hr style="margin:5px 0;">
        <b>ISO Code:</b> {row['iso_code']}<br>
        <b>Region:</b> {row['region']}<br>
        <b>Area:</b> {row['area_km2']:,.0f} km\u00b2
    </div>
    """
    
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda x, color=row['color']: {
            'fillColor': color, 'color': 'black',
            'weight': 1, 'fillOpacity': 0.5
        },
        highlight_function=lambda x: {'weight': 3, 'fillOpacity': 0.7},
        popup=folium.Popup(popup_html, max_width=250),
        tooltip=f"{row['state_name']} ({row['state_name_ar']})"
    ).add_to(m)

# Add state name markers
for _, row in gdf.iterrows():
    folium.Marker(
        location=[row['centroid_lat'], row['centroid_lon']],
        icon=folium.DivIcon(html=f'<div style="font-size:8px; font-weight:bold; white-space:nowrap;">{row["state_name"]}</div>')
    ).add_to(m)

# Add legend
legend_html = '''
<div style="position: fixed; bottom: 30px; left: 30px; z-index: 1000;
     background-color: white; padding: 10px; border-radius: 5px;
     border: 2px solid grey; font-size: 12px;">
    <b>Regions</b><br>
    <i style="background:#e74c3c; width:12px; height:12px; display:inline-block;"></i> Darfur<br>
    <i style="background:#f39c12; width:12px; height:12px; display:inline-block;"></i> Kordofan<br>
    <i style="background:#2ecc71; width:12px; height:12px; display:inline-block;"></i> Central<br>
    <i style="background:#3498db; width:12px; height:12px; display:inline-block;"></i> Eastern<br>
    <i style="background:#9b59b6; width:12px; height:12px; display:inline-block;"></i> Northern
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

m

## 11. Regional Comparison — Sudan vs Neighbors on Map

Compare GDP per capita across Sudan and its neighboring countries using a bar chart with country flags/colors.

In [ ]:
# GDP per capita comparison
gdp_pop_df = conn.sql("""
    SELECT
        p.country_name,
        p.value AS population,
        g.value AS gdp,
        ROUND(g.value / p.value, 2) AS gdp_per_capita
    FROM SUDAN_WorldBank('SP.POP.TOTL', countries := ['SDN','EGY','ETH','TCD','SSD','ERI','LBY','CAF']) p
    JOIN SUDAN_WorldBank('NY.GDP.MKTP.CD', countries := ['SDN','EGY','ETH','TCD','SSD','ERI','LBY','CAF']) g
        ON p.country = g.country AND p.year = g.year
    WHERE p.year = (SELECT MAX(year) FROM SUDAN_WorldBank('NY.GDP.MKTP.CD') WHERE value IS NOT NULL)
        AND p.value IS NOT NULL AND g.value IS NOT NULL
    ORDER BY gdp_per_capita DESC
""").df()

if len(gdp_pop_df) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    colors = ['#e74c3c' if c == 'Sudan' else '#3498db' for c in gdp_pop_df['country_name']]
    bars = ax.bar(gdp_pop_df['country_name'], gdp_pop_df['gdp_per_capita'], color=colors, alpha=0.85)
    for bar, val in zip(bars, gdp_pop_df['gdp_per_capita']):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 30,
                f'${val:,.0f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    ax.set_title('GDP Per Capita - Sudan vs Neighbors', fontsize=14, fontweight='bold')
    ax.set_ylabel('GDP per capita (USD)')
    ax.grid(True, alpha=0.3, axis='y')
    plt.xticks(rotation=30, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print('Could not compute GDP per capita (data may be unavailable for the latest year)')

## 12. Export

Export the GeoDataFrame and analysis results.

In [ ]:
# Export to GeoJSON
gdf.to_file('sudan_states.geojson', driver='GeoJSON')
print('Exported: sudan_states.geojson')

# Export to GeoPackage
gdf.to_file('sudan_states.gpkg', driver='GPKG')
print('Exported: sudan_states.gpkg')

# Export summary CSV
summary = gdf[['state_name', 'state_name_ar', 'iso_code', 'region', 'area_km2',
               'centroid_lon', 'centroid_lat']].copy()
summary.to_csv('sudan_states_summary.csv', index=False)
print('Exported: sudan_states_summary.csv')

# Save interactive map as HTML
m.save('sudan_interactive_map.html')
print('Exported: sudan_interactive_map.html')

print('\nAll exports complete! Files are in the current directory.')